Вторая версия функции для удаления нот. В этот раз одновременно звучащие ноты, например, аккорды, не будут превращаться в последовательность нот

In [9]:
import os
import pretty_midi

midi_folder = "lettersabc/a/balanced_instruments/no_pauses"

INSTRUMENT_NOTES = {
    'Accordion': (40, 108),
    'Acoustic Bass': (28, 67),
    'Acoustic Guitar _nylon_': (52, 91),
    'Acoustic Guitar _steel_': (52, 91),
    'Acoustic Grand Piano': (21, 108),
    'Alto Sax': (58, 90),
    'Bagpipe': (62, 81),
    'Banjo': (55, 79),
    'Baritone Sax': (58, 90),
    'Bassoon': (35, 77),
    'Bright Acoustic Piano': (21, 108),
    'Celesta': (60, 108),
    'Cello': (36, 81),
    'Church Organ': (36, 96),
    'Clarinet': (52, 91),
    'Contrabass': (28, 71),
    'Dulcimer': (48, 96),
    'English Horn': (48, 83),
    'Fiddle': (55, 105),
    'Flute': (60, 96),
    'French Horn': (35, 77),
    'Glockenspiel': (77, 108),
    'Guitar Harmonics': (59, 101),
    'Harmonica': (54, 96),
    'Harspichord': (29, 89),
    'Kalimba': (50, 89),
    'Marimba': (36, 96),
    'Muted Trumpet': (54, 84),
    'Oboe': (58, 89),
    'Ocarina': (55, 96),
    'Orchestral Harp': (29, 103),
    'Pan Flute': (36, 93),
    'Percussive Organ': (36, 96),
    'Piccolo': (74, 108),
    'Pizzicato Strings': (28, 100),
    'Timpani': (74, 93),
    'Tinkle Bell': (84, 115),
    'Tremolo Strings': (28, 100),
    'Trombone': (31, 77),
    'Trumpet': (60, 83),
    'Tuba': (24, 64),
    'Tubular Bells': (55, 84),
    'Vibraphone': (48, 96),
    'Viola': (48, 88),
    'Violin': (55, 105),
    'Xylophone': (53, 96)
}

# def is_valid_instrument(instrument_name):
#     base_name = instrument_name.replace('_no_pauses', '')
#     return base_name in INSTRUMENT_NOTES
#     # return instrument_name in INSTRUMENT_NOTES

def get_base_instrument_name(filename):
    # Удаляем расширение .mid и _no_pauses
    base = os.path.splitext(filename)[0]
    return base.replace('_no_pauses', '')

def is_valid_instrument(instrument_name):
    return instrument_name in INSTRUMENT_NOTES

def remove_midi_pauses_preserve_chords(midi_path, output_path=None):
    try:
        # midi = pretty_midi.PrettyMIDI(os.path.join(midi_folder, midi_path))
        # instrument_name = midi_path.split('.')[0]

        full_path = os.path.join(midi_folder, midi_path)
        
        instrument_name = get_base_instrument_name(midi_path)

        midi = pretty_midi.PrettyMIDI(full_path)

        if not is_valid_instrument(instrument_name):
            print(f"Пропущен (невалидный инструмент): {instrument_name}")
            return False

        instrument_range = INSTRUMENT_NOTES[instrument_name]
        has_valid_notes = False

        for instrument in midi.instruments:
            if not instrument.notes:
                continue

            # Оставляем только ноты в допустимом диапазоне
            valid_notes = [note for note in instrument.notes if instrument_range[0] <= note.pitch <= instrument_range[1]]
            if not valid_notes:
                continue

            has_valid_notes = True
            valid_notes.sort(key=lambda n: n.start)

            # Группируем ноты по времени старта
            grouped_notes = []
            group = [valid_notes[0]]
            delta = 0.01  # допуск на одновременность

            for note in valid_notes[1:]:
                if abs(note.start - group[-1].start) < delta:
                    group.append(note)
                else:
                    grouped_notes.append(group)
                    group = [note]
            grouped_notes.append(group)  # добавим последнюю группу

            # Перестроим группы без пауз
            current_time = 0.0
            new_notes = []
            for group in grouped_notes:
                max_duration = max(n.end - n.start for n in group)
                for note in group:
                    duration = note.end - note.start
                    note.start = current_time
                    note.end = current_time + duration
                    new_notes.append(note)
                current_time += max_duration  # сохраняем длительность аккорда

            instrument.notes = new_notes

        if not has_valid_notes:
            print(f"Пропущен (нет валидных нот): {instrument_name}")
            return False

        if output_path is None:
            base, ext = os.path.splitext(midi_path)
            output_path = os.path.join(midi_folder, 'valid', f'{base}{ext}')

        midi.write(output_path)
        print(f"✅ Успешно обработан: {instrument_name}")
        return True

    except Exception as e:
        print(f"❌ Ошибка при обработке {midi_path}: {e}")
        return False

for file in os.listdir(midi_folder):
    remove_midi_pauses_preserve_chords(file)

✅ Успешно обработан: Accordion
✅ Успешно обработан: Acoustic Bass
✅ Успешно обработан: Acoustic Grand Piano
✅ Успешно обработан: Acoustic Guitar _nylon_
✅ Успешно обработан: Acoustic Guitar _steel_
Пропущен (невалидный инструмент): Agogo
✅ Успешно обработан: Alto Sax
✅ Успешно обработан: Bagpipe
✅ Успешно обработан: Banjo
✅ Успешно обработан: Baritone Sax
✅ Успешно обработан: Bassoon
Пропущен (невалидный инструмент): Blown Bottle
Пропущен (невалидный инструмент): Brass Section
✅ Успешно обработан: Bright Acoustic Piano
✅ Успешно обработан: Celesta
✅ Успешно обработан: Cello
✅ Успешно обработан: Church Organ
✅ Успешно обработан: Clarinet
Пропущен (невалидный инструмент): Clavinet
✅ Успешно обработан: Contrabass
Пропущен (невалидный инструмент): Distortion Guitar
Пропущен (невалидный инструмент): Drawbar Organ
✅ Успешно обработан: Dulcimer
✅ Успешно обработан: English Horn
✅ Успешно обработан: Fiddle
✅ Успешно обработан: Flute
✅ Успешно обработан: French Horn
Пропущен (невалидный инструм